# Import datasets

In [6]:
import numpy as np
import pandas as pd
def load_dataset(name):
    x = np.load(f'/datasets/nicolas_facchinetti/processed_data/{name}/x_im.npy')
    y = np.load(f'/datasets/nicolas_facchinetti/processed_data/{name}/y_im.npy')
    metadata = pd.read_csv(f'/datasets/nicolas_facchinetti/processed_data/{name}/mod_metadata.csv')
    return x, y, metadata

In [10]:
datasets = ["emodb", "emovo", "ravdess"]
data = {}
for d in datasets:
    x, y, md = load_dataset(d)
    data[d] = {}
    data[d]["x"] = x
    data[d]["y"] = y
    data[d]["metadata"] = md

In [11]:
data

{'emodb': {'x': array([[[[0.        , 0.        , 0.00257353],
           [0.        , 0.        , 0.00882353],
           [0.        , 0.        , 0.00882353],
           ...,
           [0.35588235, 0.1       , 0.35588235],
           [0.5382353 , 0.16176471, 0.32352942],
           [0.52003676, 0.15514706, 0.31194854]],
  
          [[0.        , 0.        , 0.00514706],
           [0.        , 0.        , 0.01568628],
           [0.        , 0.        , 0.01568628],
           ...,
           [0.5009804 , 0.14313726, 0.50686276],
           [0.7921569 , 0.24901961, 0.44411764],
           [0.7667892 , 0.2396446 , 0.42922795]],
  
          [[0.        , 0.        , 0.00514706],
           [0.        , 0.        , 0.01568628],
           [0.        , 0.        , 0.01568628],
           ...,
           [0.4862745 , 0.13725491, 0.50980395],
           [0.8862745 , 0.3019608 , 0.4       ],
           [0.85759807, 0.29093137, 0.3867647 ]],
  
          ...,
  
          [[0.        , 0.

# Define architectures

In [1]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, LSTM, TimeDistributed, InputLayer, Reshape, BatchNormalization

2022-11-07 10:39:45.851957: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Search the best architecture

In [110]:
archs = []

In [111]:
arch0 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((16,16,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2))),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch0.summary()

archs.append(arch0.get_config())

Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_47 (Reshape)         (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization_52 (Batc (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed_324 (TimeDi (None, 16, 14, 254, 16)   448       
_________________________________________________________________
time_distributed_325 (TimeDi (None, 16, 7, 127, 16)    0         
_________________________________________________________________
time_distributed_326 (TimeDi (None, 16, 5, 125, 32)    4640      
_________________________________________________________________
time_distributed_327 (TimeDi (None, 16, 2, 62, 32)     0         
_________________________________________________________________
time_distributed_328 (TimeDi (None, 16, 3968)        

In [112]:
arch1 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((16,16,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch1.summary()

archs.append(arch1.get_config())

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_48 (Reshape)         (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization_53 (Batc (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed_329 (TimeDi (None, 16, 14, 254, 32)   896       
_________________________________________________________________
time_distributed_330 (TimeDi (None, 16, 7, 127, 32)    0         
_________________________________________________________________
time_distributed_331 (TimeDi (None, 16, 5, 125, 64)    18496     
_________________________________________________________________
time_distributed_332 (TimeDi (None, 16, 2, 62, 64)     0         
_________________________________________________________________
time_distributed_333 (TimeDi (None, 16, 7936)        

In [113]:
arch2 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((16,16,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(256, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch2.summary()

archs.append(arch2.get_config())

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_49 (Reshape)         (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization_54 (Batc (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed_334 (TimeDi (None, 16, 14, 254, 32)   896       
_________________________________________________________________
time_distributed_335 (TimeDi (None, 16, 7, 127, 32)    0         
_________________________________________________________________
time_distributed_336 (TimeDi (None, 16, 5, 125, 64)    18496     
_________________________________________________________________
time_distributed_337 (TimeDi (None, 16, 2, 62, 64)     0         
_________________________________________________________________
time_distributed_338 (TimeDi (None, 16, 7936)        

In [114]:
arch3 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((8,32,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(16, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch1.summary()

archs.append(arch1.get_config())

Model: "sequential_48"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_48 (Reshape)         (None, 16, 16, 256, 3)    0         
_________________________________________________________________
batch_normalization_53 (Batc (None, 16, 16, 256, 3)    12        
_________________________________________________________________
time_distributed_329 (TimeDi (None, 16, 14, 254, 32)   896       
_________________________________________________________________
time_distributed_330 (TimeDi (None, 16, 7, 127, 32)    0         
_________________________________________________________________
time_distributed_331 (TimeDi (None, 16, 5, 125, 64)    18496     
_________________________________________________________________
time_distributed_332 (TimeDi (None, 16, 2, 62, 64)     0         
_________________________________________________________________
time_distributed_333 (TimeDi (None, 16, 7936)        

In [115]:
arch4 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((8,32,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),
    TimeDistributed(Conv2D(128, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch4.summary()

archs.append(arch4.get_config())

Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_51 (Reshape)         (None, 8, 32, 256, 3)     0         
_________________________________________________________________
batch_normalization_56 (Batc (None, 8, 32, 256, 3)     12        
_________________________________________________________________
time_distributed_346 (TimeDi (None, 8, 30, 254, 32)    896       
_________________________________________________________________
time_distributed_347 (TimeDi (None, 8, 15, 127, 32)    0         
_________________________________________________________________
time_distributed_348 (TimeDi (None, 8, 13, 125, 64)    18496     
_________________________________________________________________
time_distributed_349 (TimeDi (None, 8, 6, 62, 64)      0         
_________________________________________________________________
time_distributed_350 (TimeDi (None, 8, 4, 60, 128)   

In [116]:
arch5 = Sequential([
    InputLayer(input_shape=(256,256,3)),
    Reshape((8,32,256,3)),
    BatchNormalization(),
    TimeDistributed(Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2))),
    TimeDistributed(Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),
    TimeDistributed(Conv2D(128, kernel_size=(3,3), activation='relu', strides=(1))),
    TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=(2))),

    TimeDistributed(Flatten()),
    LSTM(256, return_sequences=False),
    Dropout(0.2),
    Flatten(),
    Dense(7, activation='softmax')
])
arch5.summary()

archs.append(arch5.get_config())

Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_52 (Reshape)         (None, 8, 32, 256, 3)     0         
_________________________________________________________________
batch_normalization_57 (Batc (None, 8, 32, 256, 3)     12        
_________________________________________________________________
time_distributed_353 (TimeDi (None, 8, 30, 254, 32)    896       
_________________________________________________________________
time_distributed_354 (TimeDi (None, 8, 15, 127, 32)    0         
_________________________________________________________________
time_distributed_355 (TimeDi (None, 8, 13, 125, 64)    18496     
_________________________________________________________________
time_distributed_356 (TimeDi (None, 8, 6, 62, 64)      0         
_________________________________________________________________
time_distributed_357 (TimeDi (None, 8, 4, 60, 128)   

In [117]:
archs

[{'name': 'sequential_47',
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': (None, 256, 256, 3),
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input_48'}},
   {'class_name': 'Reshape',
    'config': {'name': 'reshape_47',
     'trainable': True,
     'dtype': 'float32',
     'target_shape': (16, 16, 256, 3)}},
   {'class_name': 'BatchNormalization',
    'config': {'name': 'batch_normalization_52',
     'trainable': True,
     'dtype': 'float32',
     'axis': ListWrapper([4]),
     'momentum': 0.99,
     'epsilon': 0.001,
     'center': True,
     'scale': True,
     'beta_initializer': {'class_name': 'Zeros', 'config': {}},
     'gamma_initializer': {'class_name': 'Ones', 'config': {}},
     'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}},
     'moving_variance_initializer': {'class_name': 'Ones', 'config': {}},
     'beta_regularizer': None,
     'gamma_regularizer': None,
     'beta_constraint': None,


# Search best architecturs for all datasets

In [118]:
def create_model(config):
    new_model = keras.Sequential.from_config(config)
    new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return new_model

In [119]:
def leave_one_speaker_out(data, n=5):
    speakers = data.actor.unique()
    # random select n actors as test
    selected = np.random.choice(speakers, size=n, replace=False)
    
    myCViterator = []
    for i in selected:
        trainIndices = data[data['actor']!=i ].index.values.astype(int)
        testIndices =  data[data['actor']==i ].index.values.astype(int)
        myCViterator.append((trainIndices, testIndices))
    return myCViterator

In [137]:
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

results_arch = {}
for count, a in enumerate(archs):
    print(f'Architecture {count} of {len(archs)-1}')
    
    def create_model():
        new_model = tensorflow.keras.Sequential.from_config(a)
        new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return new_model
    
    model = KerasClassifier(build_fn=create_model, epochs=30, batch_size=32, verbose=0)
          
    results_arch[f"arch{count}"] = {}

    for d in data:
        print(f"\tTraining on {d}")
        x = data[d]["x"]
        y = data[d]["y"]
        metadata = data[d]["metadata"]
        res = cross_val_score(model, x, y, cv=leave_one_speaker_out(metadata, n=3))
        results_arch[f"arch{count}"][d] = res

Architecture 0 of 5
	Training on emodb
	Training on emovo
	Training on ravdess
Architecture 1 of 5
	Training on emodb
	Training on emovo
	Training on ravdess
Architecture 2 of 5
	Training on emodb


	Training on emovo
	Training on ravdess
Architecture 3 of 5
	Training on emodb
	Training on emovo
	Training on ravdess
Architecture 4 of 5
	Training on emodb


	Training on emovo
	Training on ravdess
Architecture 5 of 5
	Training on emodb
	Training on emovo
	Training on ravdess


In [138]:
results_arch

{'arch0': {'emodb': array([0.67532468, 0.41666666, 0.69230771]),
  'emovo': array([0.234375  , 0.32786885, 0.23529412]),
  'ravdess': array([0.5       , 0.375     , 0.51428574])},
 'arch1': {'emodb': array([0.56410259, 0.4642857 , 0.4375    ]),
  'emovo': array([0.30147058, 0.23529412, 0.2734375 ]),
  'ravdess': array([0.37974682, 0.32954547, 0.40217391])},
 'arch2': {'emodb': array([0.60975611, 0.6981132 , 0.56410259]),
  'emovo': array([0.30327868, 0.17647059, 0.2647059 ]),
  'ravdess': array([0.45652175, 0.47887325, 0.33333334])},
 'arch3': {'emodb': array([0.58490568, 0.66153848, 0.54166669]),
  'emovo': array([0.26050422, 0.27419356, 0.2109375 ]),
  'ravdess': array([0.40789473, 0.51428574, 0.66666669])},
 'arch4': {'emodb': array([0.64615387, 0.63636363, 0.66037738]),
  'emovo': array([0.35245901, 0.30645162, 0.26086956]),
  'ravdess': array([0.62857145, 0.42500001, 0.43589744])},
 'arch5': {'emodb': array([0.65714288, 0.63076925, 0.56666666]),
  'emovo': array([0.27941176, 0.298

In [141]:
for arch in results_arch:
    print(arch)
    for d in results_arch[arch]:
        mean = round(np.mean(results_arch[arch][d])*100, 3)
        std = round(np.std(results_arch[arch][d])*100, 3)
        print(f'\t{d}: mean {mean}, std dev {std}')

arch0
	emodb: mean 59.477, std dev 12.613
	emovo: mean 26.585, std dev 4.386
	ravdess: mean 46.31, std dev 6.257
arch1
	emodb: mean 48.863, std dev 5.448
	emovo: mean 27.007, std dev 2.712
	ravdess: mean 37.049, std dev 3.036
arch2
	emodb: mean 62.399, std dev 5.563
	emovo: mean 24.815, std dev 5.308
	ravdess: mean 42.291, std dev 6.399
arch3
	emodb: mean 59.604, std dev 4.957
	emovo: mean 24.855, std dev 2.717
	ravdess: mean 52.962, std dev 10.62
arch4
	emodb: mean 64.763, std dev 0.986
	emovo: mean 30.659, std dev 3.739
	ravdess: mean 49.649, std dev 9.35
arch5
	emodb: mean 61.819, std dev 3.799
	emovo: mean 27.854, std dev 1.658
	ravdess: mean 35.946, std dev 4.509


# Hyperparameter optimization

In [142]:
chosen = archs[4]
def create_model_opt():
    new_model = tensorflow.keras.Sequential.from_config(chosen)
    new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return new_model
    
model = KerasClassifier(build_fn=create_model_opt, verbose=0)

In [143]:
from sklearn.model_selection import GridSearchCV
# define the grid search parameters
batch_size = [16, 32, 64, 128]
epochs = [30, 50, 70]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [145]:
results_opt = {}
for d in data:
    print(f'Dataset {d}')
    
    x = data[d]["x"]
    y = data[d]["y"]
    metadata = data[d]["metadata"]
    cv = leave_one_speaker_out(metadata, n=3)
    
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, verbose=3)
    grid_result = grid.fit(x, y)
    results_opt[d] = grid_result
    

Dataset emodb
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END ..........batch_size=16, epochs=30;, score=0.649 total time=  18.0s
[CV 2/3] END ..........batch_size=16, epochs=30;, score=0.667 total time=  19.4s
[CV 3/3] END ..........batch_size=16, epochs=30;, score=0.631 total time=  18.5s
[CV 1/3] END ..........batch_size=16, epochs=50;, score=0.662 total time=  28.2s
[CV 2/3] END ..........batch_size=16, epochs=50;, score=0.564 total time=  30.4s
[CV 3/3] END ..........batch_size=16, epochs=50;, score=0.600 total time=  29.8s
[CV 1/3] END ..........batch_size=16, epochs=70;, score=0.662 total time=  40.0s
[CV 2/3] END ..........batch_size=16, epochs=70;, score=0.615 total time=  42.0s
[CV 3/3] END ..........batch_size=16, epochs=70;, score=0.585 total time=  40.4s
[CV 1/3] END ..........batch_size=32, epochs=30;, score=0.714 total time=  15.6s
[CV 2/3] END ..........batch_size=32, epochs=30;, score=0.641 total time=  16.6s
[CV 3/3] END ..........batch_size=

[CV 2/3] END ..........batch_size=64, epochs=30;, score=0.615 total time=  18.1s
[CV 3/3] END ..........batch_size=64, epochs=30;, score=0.600 total time=  16.2s
[CV 1/3] END ..........batch_size=64, epochs=50;, score=0.636 total time=  25.2s
[CV 2/3] END ..........batch_size=64, epochs=50;, score=0.641 total time=  26.6s
[CV 3/3] END ..........batch_size=64, epochs=50;, score=0.600 total time=  25.6s
[CV 1/3] END ..........batch_size=64, epochs=70;, score=0.649 total time=  34.8s
[CV 2/3] END ..........batch_size=64, epochs=70;, score=0.615 total time=  36.5s
[CV 3/3] END ..........batch_size=64, epochs=70;, score=0.662 total time=  35.3s
[CV 1/3] END .........batch_size=128, epochs=30;, score=0.623 total time=  16.4s
[CV 2/3] END .........batch_size=128, epochs=30;, score=0.667 total time=  15.4s
[CV 3/3] END .........batch_size=128, epochs=30;, score=0.554 total time=  15.0s


[CV 1/3] END .........batch_size=128, epochs=50;, score=0.662 total time=  23.3s
[CV 2/3] END .........batch_size=128, epochs=50;, score=0.641 total time=  24.7s
[CV 3/3] END .........batch_size=128, epochs=50;, score=0.600 total time=  23.5s
[CV 1/3] END .........batch_size=128, epochs=70;, score=0.649 total time=  31.8s
[CV 2/3] END .........batch_size=128, epochs=70;, score=0.692 total time=  35.1s
[CV 3/3] END .........batch_size=128, epochs=70;, score=0.662 total time=  32.3s
Dataset emovo
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END ..........batch_size=16, epochs=30;, score=0.227 total time=  21.2s
[CV 2/3] END ..........batch_size=16, epochs=30;, score=0.311 total time=  21.1s
[CV 3/3] END ..........batch_size=16, epochs=30;, score=0.250 total time=  20.7s
[CV 1/3] END ..........batch_size=16, epochs=50;, score=0.286 total time=  33.9s
[CV 2/3] END ..........batch_size=16, epochs=50;, score=0.361 total time=  33.5s
[CV 3/3] END ..........batch_size=

[CV 1/3] END ..........batch_size=64, epochs=50;, score=0.252 total time=  29.1s
[CV 2/3] END ..........batch_size=64, epochs=50;, score=0.352 total time=  29.3s
[CV 3/3] END ..........batch_size=64, epochs=50;, score=0.258 total time=  28.6s
[CV 1/3] END ..........batch_size=64, epochs=70;, score=0.252 total time=  40.7s
[CV 2/3] END ..........batch_size=64, epochs=70;, score=0.369 total time=  40.7s
[CV 3/3] END ..........batch_size=64, epochs=70;, score=0.266 total time=  41.4s
[CV 1/3] END .........batch_size=128, epochs=30;, score=0.328 total time=  17.6s
[CV 2/3] END .........batch_size=128, epochs=30;, score=0.361 total time=  17.6s
[CV 3/3] END .........batch_size=128, epochs=30;, score=0.305 total time=  17.3s
[CV 1/3] END .........batch_size=128, epochs=50;, score=0.252 total time=  27.2s
[CV 2/3] END .........batch_size=128, epochs=50;, score=0.320 total time=  26.9s


[CV 3/3] END .........batch_size=128, epochs=50;, score=0.305 total time=  26.6s
[CV 1/3] END .........batch_size=128, epochs=70;, score=0.269 total time=  37.2s
[CV 2/3] END .........batch_size=128, epochs=70;, score=0.311 total time=  38.2s
[CV 3/3] END .........batch_size=128, epochs=70;, score=0.273 total time=  36.7s
Dataset ravdess
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END ..........batch_size=16, epochs=30;, score=0.316 total time=  46.9s
[CV 2/3] END ..........batch_size=16, epochs=30;, score=0.433 total time=  52.8s
[CV 3/3] END ..........batch_size=16, epochs=30;, score=0.486 total time=  52.4s
[CV 1/3] END ..........batch_size=16, epochs=50;, score=0.354 total time= 1.3min
[CV 2/3] END ..........batch_size=16, epochs=50;, score=0.500 total time= 1.5min
[CV 3/3] END ..........batch_size=16, epochs=50;, score=0.403 total time= 1.4min
[CV 1/3] END ..........batch_size=16, epochs=70;, score=0.329 total time= 1.8min
[CV 2/3] END ..........batch_siz

[CV 1/3] END ..........batch_size=32, epochs=70;, score=0.329 total time= 1.7min
[CV 2/3] END ..........batch_size=32, epochs=70;, score=0.433 total time= 1.7min
[CV 3/3] END ..........batch_size=32, epochs=70;, score=0.444 total time= 1.7min
[CV 1/3] END ..........batch_size=64, epochs=30;, score=0.392 total time=  45.8s
[CV 2/3] END ..........batch_size=64, epochs=30;, score=0.483 total time=  47.9s
[CV 3/3] END ..........batch_size=64, epochs=30;, score=0.514 total time=  45.8s
[CV 1/3] END ..........batch_size=64, epochs=50;, score=0.291 total time= 1.2min
[CV 2/3] END ..........batch_size=64, epochs=50;, score=0.450 total time= 1.3min
[CV 3/3] END ..........batch_size=64, epochs=50;, score=0.514 total time= 1.2min
[CV 1/3] END ..........batch_size=64, epochs=70;, score=0.342 total time= 1.7min


[CV 2/3] END ..........batch_size=64, epochs=70;, score=0.533 total time= 1.7min
[CV 3/3] END ..........batch_size=64, epochs=70;, score=0.458 total time= 1.8min
[CV 1/3] END .........batch_size=128, epochs=30;, score=0.380 total time=  42.4s
[CV 2/3] END .........batch_size=128, epochs=30;, score=0.533 total time=  42.4s
[CV 3/3] END .........batch_size=128, epochs=30;, score=0.514 total time=  42.3s
[CV 1/3] END .........batch_size=128, epochs=50;, score=0.468 total time= 1.1min
[CV 2/3] END .........batch_size=128, epochs=50;, score=0.450 total time= 1.1min
[CV 3/3] END .........batch_size=128, epochs=50;, score=0.472 total time= 1.1min
[CV 1/3] END .........batch_size=128, epochs=70;, score=0.430 total time= 1.6min
[CV 2/3] END .........batch_size=128, epochs=70;, score=0.450 total time= 1.6min
[CV 3/3] END .........batch_size=128, epochs=70;, score=0.514 total time= 1.6min


In [146]:
# summarize results
for d in results_opt:
    print(d)
    grid_result = results_opt[d]
    print("\tBest: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("\t%f (%f) with: %r" % (mean, stdev, param))

emodb
	Best: 0.677411 using {'batch_size': 32, 'epochs': 30}
	0.648929 (0.014658) with: {'batch_size': 16, 'epochs': 30}
	0.608813 (0.040586) with: {'batch_size': 16, 'epochs': 50}
	0.620779 (0.031958) with: {'batch_size': 16, 'epochs': 70}
	0.677411 (0.029910) with: {'batch_size': 32, 'epochs': 30}
	0.666245 (0.028795) with: {'batch_size': 32, 'epochs': 50}
	0.654168 (0.015106) with: {'batch_size': 32, 'epochs': 70}
	0.638894 (0.044571) with: {'batch_size': 64, 'epochs': 30}
	0.625796 (0.018340) with: {'batch_size': 64, 'epochs': 50}
	0.642091 (0.019529) with: {'batch_size': 64, 'epochs': 70}
	0.614630 (0.046472) with: {'batch_size': 128, 'epochs': 30}
	0.634454 (0.025870) with: {'batch_size': 128, 'epochs': 50}
	0.667732 (0.018076) with: {'batch_size': 128, 'epochs': 70}
emovo
	Best: 0.331025 using {'batch_size': 128, 'epochs': 30}
	0.262789 (0.035696) with: {'batch_size': 16, 'epochs': 30}
	0.309207 (0.036426) with: {'batch_size': 16, 'epochs': 50}
	0.257669 (0.008121) with: {'batch